In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/home/laurent/quantic/science/HomeHots/HOTS_clone_laurent/HOTS


In [2]:
import os
import pickle
import datetime
from mix_Network import *
download = False
learn_set = tonic.datasets.NMNIST(save_to='../Data/',
                                  train=True, download=download,
                                  transform=tonic.transforms.AERtoVector()
                                 )
records_path = '../Records'
timestr = datetime.datetime.now().date().isoformat()
timestr = '2021-02-13'
verbose = True

%mkdir -p ../Records
%mkdir -p ../Records/EXP_03_NMNIST

homeo = True
sigma = None
pooling = False
homeinv = False
jitter = False
tau = 5
krnlinit = 'first'
nblay = 3
nbclust = 4

ds = 3/2
ds = 75
nb_train = int(7500//ds)
nb_test = int(2500//ds)

Using downloaded and verified file: ../Data/nmnist_train.zip
Extracting ../Data/nmnist_train.zip to ../Data/


In [3]:
import time
def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')

In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader
torch.set_default_tensor_type("torch.DoubleTensor") # -> torch.tensor([1.2, 3]).dtype = torch.float64
# https://sebastianraschka.com/faq/docs/pytorch-crossentropy.html
#criterion = torch.nn.NLLLoss(reduction="mean") # loss divided by output size
criterion = torch.nn.BCELoss(reduction="mean") # loss divided by output size

class LogisticRegressionModel(torch.nn.Module):
    #torch.nn.Module -> Base class for all neural network modules
    def __init__(self, N, n_classes, bias=True):
        super(LogisticRegressionModel, self).__init__() 
        self.linear = torch.nn.Linear(N, n_classes, bias=bias)
        self.nl = torch.nn.Softmax(dim=1)

    def forward(self, factors):
        return self.nl(self.linear(factors))

In [5]:
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 5 + 1
#num_epochs = 2 ** 9 + 1
# batch_size = 256
n_classes=10
amsgrad = False # gives similar results
amsgrad = True  # gives similar results

def fit_raw_data(dataset, 
            nb_digit,
            learning_rate=learning_rate,
            # batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=False, #**kwargs
        ):
    

    generator = torch.Generator().manual_seed(42)
    sampler = torch.utils.data.RandomSampler(dataset, replacement=True, num_samples=nb_digit, generator=generator)
    loader = tonic.datasets.DataLoader(dataset, sampler=sampler)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'device -> {device}')

    N = 34*34*2
    n_classes = 10
    logistic_model = LogisticRegressionModel(N, n_classes)
    logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(
        logistic_model.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
    )
    
    for epoch in range(int(num_epochs)):
        losses = []
        for X, label in loader:
            X, label = X.to(device), label.to(device)
            X, label = X.squeeze(0), label.squeeze(0) # just one digit = one batch
            outputs = logistic_model(X)

            n_events = X.shape[0]
            #print(X.squeeze(0).shape, label * torch.ones((1, n_events)))
            #print(outputs, label)
            labels = label*torch.ones(n_events).type(torch.LongTensor).to(device)
            labels = torch.nn.functional.one_hot(labels, num_classes=n_classes).type(torch.DoubleTensor).to(device)
            #print(outputs.shape, labels.shape)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        if verbose and (epoch % (num_epochs // 32) == 0):
            print(f"Iteration: {epoch} - Loss: {np.mean(losses):.5f}")
            
    return logistic_model, losses

In [6]:
tic()
model, loss = fit_raw_data(learn_set, 
            nb_train,
            learning_rate=learning_rate,
            # batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=True,
        )
toc()

device -> cuda
Iteration: 0 - Loss: 0.42352
Iteration: 1 - Loss: 0.41369
Iteration: 2 - Loss: 0.26805
Iteration: 3 - Loss: 0.32691
Iteration: 4 - Loss: 0.23339
Iteration: 5 - Loss: 0.21581
Iteration: 6 - Loss: 0.25537
Iteration: 7 - Loss: 0.27334
Iteration: 8 - Loss: 0.18722
Iteration: 9 - Loss: 0.25079
Iteration: 10 - Loss: 0.31585
Iteration: 11 - Loss: 0.27489
Iteration: 12 - Loss: 0.25002
Iteration: 13 - Loss: 0.25090
Iteration: 14 - Loss: 0.28778
Iteration: 15 - Loss: 0.30497
Iteration: 16 - Loss: 0.19089
Iteration: 17 - Loss: 0.31607
Iteration: 18 - Loss: 0.32918
Iteration: 19 - Loss: 0.26271
Iteration: 20 - Loss: 0.28166
Iteration: 21 - Loss: 0.20387
Iteration: 22 - Loss: 0.16766
Iteration: 23 - Loss: 0.25229
Iteration: 24 - Loss: 0.29053
Iteration: 25 - Loss: 0.30063
Iteration: 26 - Loss: 0.27631
Iteration: 27 - Loss: 0.21539
Iteration: 28 - Loss: 0.24402
Iteration: 29 - Loss: 0.34751
Iteration: 30 - Loss: 0.21999
Iteration: 31 - Loss: 0.22052
Iteration: 32 - Loss: 0.25277
Done 

In [7]:
test_set = tonic.datasets.NMNIST(save_to='../Data/',
                                train=False, download=download,
                                transform=tonic.transforms.AERtoVector())

Using downloaded and verified file: ../Data/nmnist_test.zip
Extracting ../Data/nmnist_test.zip to ../Data/


In [8]:
def predict_data(test_set, model, # gamma=gamma,
            verbose=False, **kwargs
        ):
    
    with torch.no_grad():

        generator=torch.Generator().manual_seed(42)
        sampler = torch.utils.data.RandomSampler(test_set, replacement=True, num_samples=nb_test, generator=generator)
        loader = tonic.datasets.DataLoader(test_set, sampler=sampler)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        logistic_model = model.to(device)

        pred_target, true_target = [], []

        for X, label in loader:
            X = X.to(device)
            X, label = X.squeeze(0), label.squeeze(0)

            n_events = X.shape[0]
            labels = label*torch.ones(n_events).type(torch.LongTensor)

            outputs = logistic_model(X)

            pred_target.append(torch.argmax(outputs, dim=1).cpu().numpy())
            true_target.append(labels.numpy())

    return pred_target, true_target

In [9]:
pred_target, true_target = predict_data(test_set, model)

In [10]:
pred_target, true_target

([array([1, 1, 1, ..., 7, 7, 7]),
  array([1, 1, 1, ..., 6, 6, 6]),
  array([1, 1, 1, ..., 6, 6, 6]),
  array([1, 1, 1, ..., 6, 6, 6]),
  array([1, 1, 4, ..., 0, 0, 0]),
  array([1, 1, 1, ..., 3, 3, 3]),
  array([1, 1, 1, ..., 3, 3, 3]),
  array([1, 1, 1, ..., 1, 1, 1]),
  array([1, 1, 1, ..., 8, 8, 8]),
  array([1, 7, 7, ..., 7, 7, 7]),
  array([1, 1, 1, ..., 3, 3, 3]),
  array([1, 1, 1, ..., 9, 9, 9]),
  array([1, 7, 1, ..., 7, 7, 7]),
  array([1, 7, 7, ..., 3, 3, 3]),
  array([1, 7, 7, ..., 3, 3, 3]),
  array([1, 1, 3, ..., 7, 7, 7]),
  array([1, 1, 1, ..., 7, 7, 7]),
  array([1, 1, 1, ..., 3, 3, 3]),
  array([1, 1, 1, ..., 8, 8, 8]),
  array([1, 4, 1, ..., 8, 8, 8]),
  array([1, 1, 1, ..., 9, 9, 9]),
  array([1, 1, 1, ..., 8, 8, 8]),
  array([1, 1, 1, ..., 1, 1, 1]),
  array([1, 1, 1, ..., 1, 1, 1]),
  array([1, 1, 7, ..., 3, 3, 3]),
  array([1, 1, 1, ..., 3, 3, 3]),
  array([1, 1, 1, ..., 9, 9, 9]),
  array([1, 5, 0, ..., 4, 4, 4]),
  array([1, 1, 0, ..., 2, 2, 2]),
  array([1, 1,

In [11]:
for pred_target_, true_target_ in zip(pred_target, true_target):
    print(len(pred_target_), len(true_target_))
    print(len(pred_target_ == true_target_))
    print(np.mean((pred_target_ == true_target_)*1.))

3808 3808
3808
0.8022584033613446
4645 4645
4645
0.9898815931108719
3595 3595
3595
0.9955493741307372
4304 4304
4304
0.9818773234200744
5272 5272
5272
0.9779969650986343
4337 4337
4337
0.301129813234955
4947 4947
4947
0.5476046088538509
1598 1598
1598
1.0
3659 3659
3659
0.5649084449303088
3479 3479
3479
0.9833285426846795
2483 2483
2483
0.873942811115586
4235 4235
4235
0.9506493506493506
3379 3379
3379
0.9949689257176679
4791 4791
4791
0.0002087246921310791
5527 5527
5527
0.9714130631445631
4186 4186
4186
0.9581939799331104
4492 4492
4492
0.9165182546749777
4006 4006
4006
0.654018971542686
5346 5346
5346
0.9784885895997008
3102 3102
3102
0.6963249516441006
4481 4481
4481
0.9848248158893104
4491 4491
4491
0.7252282342462704
2734 2734
2734
0.9959765910753475
2880 2880
2880
1.0
6213 6213
6213
0.9930790278448415
4953 4953
4953
0.9852614577024026
3396 3396
3396
0.9967608951707891
3452 3452
3452
0.9727694090382387
4508 4508
4508
0.9924578527062999
4508 4508
4508
0.9966725820763088
5550 5550


In [12]:
accuracy = []
for pred_target_, true_target_ in zip(pred_target, true_target):
    accuracy.append(np.mean(pred_target_ == true_target_))
print(f'{np.mean(accuracy)=}')    

np.mean(accuracy)=0.8424701874026173
